# Full example - full automatization optimization engine
- Hardcoded optimizer creating each variable manually
- Hardcoded optimizer creating each constraints manually
- There are created configuration tables in excel tables that the optimizer read
- IMPORTANT: the observed variables in this code is a decision variable in optimization but its value is fixed

About observed variables (that always have a fixed value)
- The observed variables are not decision variables, but with the idea to automatize this codes they are defined as decision variables and then a constraint is added to set its values to a unique values

- Doing this is a little automatice the creation of the constraints and open the posibility to transform the observed variables into a decision variables and connect other models/tanks/process to this observed variables with a less modification in the codes

- This example needs a gurobi licence

## Root folder and read env variables

In [ ]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## Load licence gurobi

In [ ]:
##########  LOAD LICENCE GUROBI ##########
import gurobipy as gp

# set env variable with the path of the licence
name_file_licence_gurobi = "gurobi.lic"
path_licence_gurobi = root_path + '\\' + name_file_licence_gurobi
os.environ ["GRB_LICENSE_FILE"] = path_licence_gurobi
print(os.environ["GRB_LICENSE_FILE"])

In [ ]:
######### LAOD CONTENT LICENCE GUROBI #########
with open(path_licence_gurobi, 'r') as f:
    content_licence = f.read()
WLSACCESSID = content_licence.split('\n')[3].split('=')[1] # load WLSACCESSID (string)
WLSSECRET = content_licence.split('\n')[4].split('=')[1] # load WLSSECRET (string)
LICENSEID = int(content_licence.split('\n')[5].split("=")[1]) # load LICENSEID (integer)

params = {
"WLSACCESSID": WLSACCESSID,
"WLSSECRET": WLSSECRET,
"LICENSEID": LICENSEID
}

## RUN

In [ ]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### 1. Load configuration file optimizer and configuration instance to solve
The are principally two kind of files to config optimizer
- **configuration file to create optimizer**: there a files used to create the optimization network such as, list of sets, list of variables, upper bound and lower bound, etc. Pricipally in this files should be parameters that doesn't change too much across the time. For example, the list of variable, is a parameter that if change, the structure of the network change and the machine learning models needs to change too

- **configuration file with instance to solve**: there files that change its values every time that the optimizer solve the problem. It represents the files with the actual values of the features, and so, this values change every time that the optimizer is executed

#### 1.1. IndexTime file

In [ ]:
#################### define set ####################

## paths and files names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_indextime = 'IndexTime.xlsx'
path_indextime = path_folder_config_optimization + file_indextime

# read file
indextime = pd.read_excel(path_indextime)

# set index
index_set_time = pd.Index(indextime['IndexTime'].values)
index_set_time

#### 1.2 Decision variables

In [ ]:
#################### define decision variables ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_allvariables = 'AllVariables.xlsx'
path_allvariables = path_folder_config_optimization + file_allvariables

# read file
config_allvariables = pd.read_excel(path_allvariables)

# table
config_allvariables

#### 1.3 Initial Values
-  This configuration files corresponde to the description **"configuration file with instance to solve"**. This file has the initial values to start the optimizer
-  **OBS: the decision variable that are targets of machine learning models its initial value it nos defined. For all the optimization process since t=0 to t=N all the values are predicted by ml model**

In [ ]:
#################### define initial values ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_initvalues = 'InitialValues.xlsx'
path_initvalues = path_folder_config_optimization + file_initvalues

# read file
config_initvalues = pd.read_excel(path_initvalues)

# table
config_initvalues

#### OBS: at this part all the parameters of decision variables and observed variables were defined. Now, it is necesary define a structure of the optimization network to have the capacity to generate whatever network of this kind of problem with the posibilty to change the number of variables, its limits, capacity of tanks and also the NUMBER of process and tanks

#### 1.4 Define models to load
In this file are defined the path to ml models used in each process. Then reading the table the optimizer can go to load the ml model

**As you can see a process can have multiple models because the process has multiple outputs. BUT each model to develop has its own and unique name**

In [ ]:
#################### define initial values ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_modelsml = 'ModelsML.xlsx'
path_modelsml = path_folder_config_optimization + file_modelsml

# read file
config_modelsml = pd.read_excel(path_modelsml)

# table
config_modelsml

#### 1.5 Map tanks
Read a configuration table that map each tank in the process. For each tank is mapped the input flows and the output flows

Important, obviosly, the input and outputs of the tank needs to be defined in the table that maps all the features in the network

In [ ]:
#################### define map input outputs each tank ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_maptanks = 'MapTanks.xlsx'
path_maptanks = path_folder_config_optimization + file_maptanks

# read file
config_maptanks = pd.read_excel(path_maptanks)

# table
config_maptanks

#### 1.6 Map process Machine learning models features and target
When each machine learning model is trained, one of the outputs of the tranining process is the artifact with the model and also a master table with the features and target of the model.

- **In this example the individual master table for each model was generated automatically and the sintetized table is generated manually by the author**

- **As you can see a process can have multiple models because the process has multiple outputs. BUT each model to develop has its own and unique name**

In [ ]:
#################### define map features and targer of each model for each process ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_mapprocess_mlmodels = 'MapProcessMLmodels.xlsx'
path_mapprocess_mlmodels = path_folder_config_optimization + file_mapprocess_mlmodels

# read filemapprocess_mlmodels
config_mapprocess_mlmodels = pd.read_excel(path_mapprocess_mlmodels)

# table
config_mapprocess_mlmodels

### 2. Load ML models
Automatically load all the models and save its into a dictionary to call when it is neccesary (as a process can have multiple models, id of the dictionary is the **name_process_model** that indicates the name for a unique model in the optimization network)
- Process A - y1
- Process B - y2
- Process C - y2 (custom model)
- Process B - y3

In [ ]:
config_modelsml

In [ ]:
# for: for each model upload this pkl and save it into a dictionary

# define a dictionary where the differents ml models are uploaded. the key of the names models is the column "name_process_model" (known the process
# and the models)
models_ml = {}
for index_modelml in range(len(config_modelsml)):

    # get the name (ID) of ml models
    config_names_modelsml = config_modelsml.loc[index_modelml, 'name_process_model']
    print('\nname model - model id: ', config_names_modelsml)

    # define parameters to laod model. path (folder) and artifact (name file)
    path_folder = config_modelsml[config_modelsml['name_process_model'] == config_names_modelsml]['path_folder'].values[0]
    artifact_name = config_modelsml[config_modelsml['name_process_model'] == config_names_modelsml]['artifact_name'].values[0]
    extension_artifact = artifact_name.split('.')[-1]
    
    # define full path to model
    path_folder_model = f'artifacts/models/{path_folder}/'
    path_model_loaded = path_folder_model + artifact_name
    print('path model loade: ', path_folder)
    print('artifact model loaded: ', artifact_name)
    print('full path model: ', path_model_loaded)
    
    # load model pkl
    if extension_artifact == 'pkl':
        model_loaded = pd.read_pickle(path_model_loaded)
        print('loaded pkl')

    # load model excel - custom model
    if extension_artifact == 'xlsx':
        model_loaded = pd.read_excel(path_model_loaded)
        print('loaded excel')

    if extension_artifact == 'h5':
        print('keras - pass - todo')
        print('loaded h5')

    # save the model loaded into a dict
    models_ml[config_names_modelsml] = model_loaded

In [ ]:
# model_process_a_y1
# model_process_b_y2
# model_process_b_y3
# model_process_c

### 2. Create gurobi model

In [ ]:
# # create model with licence
env = gp.Env(params=params)
model_opt = gp.Model('Example Optimization Model', env = env)

# create model without licence
# model_opt = gp.Model('Example Optimization Model')

### 3. Create decision variables
It is necesary to have:
- list of elements of the sets of decision var
- table with the list of decision variables to create with important considerations when the are created (for example, upper bound and lower bound)
- **create the decision var and save it into a dictionary of decision var. Use the dictionary to call the decision var when they are used to create a constraints**

#### Aditional, when the decision var is created, set the initial value

- Fix the values of period t=0 for each decision var.

- t=0 represent the actual period or initial period and it in some problems and modelations is kwown

- In addition in this notebook, the values in time t=0 are fixed for all decision variables, inclusive if the decision var have a constraint that define its values in time t = 0 (so, this kind of constraints needs to be defined since t = 1)

In [ ]:
##### define a for across the configuration table to create the decision vars and save it into a python dictionary

decision_var = {}
for index_var in range(len(config_allvariables)):

    # get config values
    config_names_decision_var = config_allvariables.loc[index_var, 'feature_name']
    config_description_decision_var = config_allvariables.loc[index_var, 'feature_name'] # use the feature name as a description in gurobi description
    print('defining decision variables: ', config_names_decision_var)

    # create decision var and save in the dictionary
    decision_var[config_names_decision_var] = gppd.add_vars(model_opt, 
                                                            index_set_time, 
                                                            name = config_description_decision_var,
                                                            #lb = -gp.GRB.INFINITY,
                                                            ub = gp.GRB.INFINITY
                                                           )

### 4. Set initial values decision variables
Set the initial values t=0 for the decision variables that needs this initial values and save in t=0

In [ ]:
# initial values decision variables - filter configuration file with only the decision var that have defined its initial values 
# (it should be all except target variables)
config_initvalues_init = config_initvalues[config_initvalues['init_values'].isnull() == False]
config_initvalues_init = config_initvalues_init.reset_index().drop(columns = 'index')
config_initvalues_init

In [ ]:
for index_var in range(len(config_initvalues_init)):

    # get config values
    config_names_decision_var = config_initvalues_init.loc[index_var, 'feature_name']
    print('set initial values decision variables: ', config_names_decision_var)

    # get initial value
    initial_value_decision_var = config_initvalues_init[config_initvalues_init['feature_name'] == config_names_decision_var]['init_values'].values[0]
    
    # set initial value t=0 for all decision variables that needs this value
    model_opt.addConstr(decision_var[config_names_decision_var]['t0'] == initial_value_decision_var,  
                        name = f'Initial Value {config_names_decision_var}')

### 5. Lower bound and Upper bound decision variables
For example, lower bound and upper bound of tank level

---
If one decision variables doesn't need a lower bound and upper bound parameter, its value in configuration file is saved as np.NaN

Then filter the configuration table to have only the decision variables that have defined a rate_change

define in differents files one for lower bounds and another for upper bounds

#### 5.1 Lower bound
Since config decision var create a subtable with only the features that have defined its upper bound and then create the constraints

In [ ]:
# lower bounds parameters - filter configuration file with only the decision var that have defined its lower bounds
config_allvariables_lower_bounds = config_allvariables[config_allvariables['lower'].isnull() == False]
config_allvariables_lower_bounds = config_allvariables_lower_bounds.reset_index().drop(columns = 'index') # reset index to count by index
config_allvariables_lower_bounds

In [ ]:
# generate constaint - lower bound

for index_var in range(len(config_allvariables_lower_bounds)):

    # get config values
    config_names_decision_var = config_allvariables_lower_bounds.loc[index_var, 'feature_name']
    print('lower bound decision_var: ', config_names_decision_var)
    
    gppd.add_constrs(model_opt, 
                     decision_var[config_names_decision_var],  # decision var
                     gp.GRB.GREATER_EQUAL,
                     config_allvariables_lower_bounds[config_allvariables_lower_bounds['feature_name'] == config_names_decision_var]['lower'].values[0],  # lower bound
                     name = f'Lower bound {config_names_decision_var}')

#### 5.2 upper bound
Since config decision var create a subtable with only the features that have defined its upper bound and then create the constraints

In [ ]:
# upper bounds parameters - filter configuration file with only the decision var that have defined its upper bounds
config_allvariables_upper_bounds = config_allvariables[config_allvariables['upper'].isnull() == False]
config_allvariables_upper_bounds = config_allvariables_upper_bounds.reset_index().drop(columns = 'index')
config_allvariables_upper_bounds

In [ ]:
# generate constaint - upper bound

for index_var in range(len(config_allvariables_upper_bounds)):

    # get config values
    config_names_decision_var = config_allvariables_upper_bounds.loc[index_var, 'feature_name']
    print('upper bound decision_var: ', config_names_decision_var)
    
    gppd.add_constrs(model_opt, 
                     decision_var[config_names_decision_var],  # decision var
                     gp.GRB.LESS_EQUAL, 
                     config_allvariables_upper_bounds[config_allvariables_upper_bounds['feature_name'] == config_names_decision_var]['upper'].values[0],  # upper bound
                     name = f'Upper bound {config_names_decision_var}')

### 6. Rate change of decision variables across the time

If one decision variables doesn't need a rate change parameter, its value in configuration file is saved as np.NaN

Then filter the configuration table to have only the decision variables that have defined a rate_change

\begin{align}
&| ~ X^{t}_{i} - X^{t-1}_{i} ~ | ~ \leq  ~ c_{i} &\quad \forall ~ i \in I, t \in T \tag{6}\\
\end{align}

#### 6.1 define table with values of rate_change

In [ ]:
# rate change parameters - filter configuration file with only the decision var that have defined its rates changes
config_allvariables_rate_change = config_allvariables[config_allvariables['rate_change'].isnull() == False]
config_allvariables_rate_change = config_allvariables_rate_change.reset_index().drop(columns = 'index')
config_allvariables_rate_change

#### 6.2 The rate change constraints is defined using absolute values. So it is necesary create an auxiliar decision variable


This auxiliar decision var created represents the difference between the decisions vars. It is necesary create auxiliar variables and fixed it first value into zero (set initial value (diff t = 0 and t = -1) is set to cero because t = -1 is not defined)

In this example
- diff_time_x >= (x(t-1) - x(t))

- diff_time_x >= -(x(t-1) - x(t))

- diff_time_x <= delta_x

In [ ]:
### create an auxiliar decion var "diff" for each decision var that has defined its rate change

aux_decision_var = {}
for index_var in range(len(config_allvariables_rate_change)):

    # get config values
    config_names_decision_var = config_allvariables_rate_change.loc[index_var, 'feature_name']
    print('create auxiliar variable diff "t" - "t-1": ', config_names_decision_var)

    # create decision var and save in the dictionary
    aux_decision_var[config_names_decision_var] = gppd.add_vars(model_opt, 
                                                                index_set_time, 
                                                                name = f'diff "t" - "t_1" of decision var: {config_names_decision_var}',
                                                                lb = -gp.GRB.INFINITY,
                                                                ub = gp.GRB.INFINITY
                                                               )

    # set initial value (diff t = 0 and t = -1) is set to cero because t = -1 is not defined
    model_opt.addConstr(aux_decision_var[config_names_decision_var]['t0'] == 0,  name = f'Initial Value diff {config_names_decision_var}')

#### 6.3 Define rate change variable constraint for each decision variable and for each time

In [ ]:
# for each variable
for index_var in range(len(config_allvariables_rate_change)):

    # get config values
    config_names_decision_var = config_allvariables_rate_change.loc[index_var, 'feature_name']
    print('rate change decision var: ', config_names_decision_var)

    # get rate change for this decision var
    rate_change_decision_var = config_allvariables_rate_change[config_allvariables_rate_change['feature_name'] == config_names_decision_var]['rate_change'].values[0]
    
    # for each time in this decision variable
    for index_time in range(1, len(index_set_time)):
        
        ### define time t and t-1
        time_t = index_set_time[index_time]
        time_t_1 = index_set_time[index_time-1]
    
        ### define constraints
        # positive segment
        model_opt.addConstr(aux_decision_var[config_names_decision_var][time_t] >= (decision_var[config_names_decision_var][time_t] - decision_var[config_names_decision_var][time_t_1]), 
                            name = f'diff {config_names_decision_var} positive segment {time_t} - {time_t_1}')

        # negative segment
        model_opt.addConstr(aux_decision_var[config_names_decision_var][time_t] >= -(decision_var[config_names_decision_var][time_t] - decision_var[config_names_decision_var][time_t_1]), 
                            name = f'diff {config_names_decision_var} negative segment {time_t} - {time_t_1}')

        # rate change
        model_opt.addConstr(aux_decision_var[config_names_decision_var][time_t] <= rate_change_decision_var, 
                            name = f'diff_var_X1 delta {time_t} - {time_t_1}')

### 7. Volumen change (Inventory change) - across time - relation between decision variablesend{align}

The only constraint that needs to be defined is (because the constraints of rate change are defined above - The two first contraints can be replaced with the constraint that defined the lower and upper bound of the decision variables.)

\begin{align}
&V^{t-1}_{j} ~ + \sum_{i}  Y^{t}_{i,j} -  \sum_{i}  Z^{t}_{i',j} ~ =  ~ V^{t}_{j} &\quad \forall ~ t \in T, j \in J \tag{4}\\
\end{align}

#### 7.1 Define full constraints - automatically all times
- Automatization constraint. For across all the set time (t0, t1, t2, etc). But it is important to see that the first value of the decision variable (period t=0) is defined before, so the "for cicle" start from (t1, t2, etc)

original codes
    
    model_opt.addConstr(var_tank_level[time_t_1] + var_Y1[time_t] - var_Z1[time_t] == var_tank_level[time_t], 
                name = f'new level of tank {time_t} - {time_t_1}')

In [ ]:
#### EXAMPLE FOR ONE TANK HARCODED

# for index_time in range(1, len(index_set_time)):
    
#     ### define time t and t-1
#     time_t = index_set_time[index_time]
#     time_t_1 = index_set_time[index_time-1]

#     ### define constraints
#     model_opt.addConstr(decision_var['TL1'][time_t_1] + decision_var['Y1'][time_t] - decision_var['Z1'][time_t] == decision_var['TL1'][time_t], 
#                 name = f'new level of tank {time_t} - {time_t_1}')

In [ ]:
# generate a list of tanks
list_tanks = config_maptanks['tank'].unique().tolist()
list_tanks

In [ ]:
# filter by each tank and build the constraint creating left side of constraint according the number of inputs and outputs flows


# FOR EACH TANK
for name_tank in list_tanks:
    print('tank name: ', name_tank)

    # get a config map tanks dataframe with only the information of the tank that is consulting
    aux_config_maptanks = config_maptanks[config_maptanks['tank'] == name_tank]


    # FOR EACH TIME CONSTRAINT
    for index_time in range(1, len(index_set_time)):
        
        ### define time t and t-1
        time_t = index_set_time[index_time]
        time_t_1 = index_set_time[index_time-1]
    
        
        # BUILD THE LEFT CONSTRAINT WITH MULTIPLE INPUT AND OUTPUT FLOWS. ALSO IDENTIFY THE RIGHT SIDE CONSTRAINT
        left_side_constraint = 0
        for tag_related_tank in aux_config_maptanks['feature_name'].to_list():
    
            # filter the configuration file only for the tag that is consulting
            aux_one_row_config_maptanks = aux_config_maptanks[aux_config_maptanks['feature_name'] == tag_related_tank]
    
            # identify if the tag is level variable adn the build the left side of constraint. also save the value to right side constraint
            if (aux_one_row_config_maptanks["input_output"] == 'L').values[0]:
                left_side_constraint += decision_var[tag_related_tank][time_t_1]
                right_side_constraint = decision_var[tag_related_tank][time_t]
    
    
            # identify if the tag is level variable adn the build the left side of constraint
            if (aux_one_row_config_maptanks["input_output"] == 'IN').values[0]:
                left_side_constraint += decision_var[tag_related_tank][time_t]
                
                
            # identify if the tag is level variable adn the build the left side of constraint
            if (aux_one_row_config_maptanks["input_output"] == 'OUT').values[0]:
                left_side_constraint -= decision_var[tag_related_tank][time_t]



        ### define constraints
        model_opt.addConstr(left_side_constraint == right_side_constraint, 
                    name = f'new level of tank {name_tank}  in times: {time_t} - {time_t_1}')


        # # debugigng
        # model_opt.update()
        # print(left_side_constraint)
        # print(right_side_constraint)

### 8. Set values of observed variables
The observed variables are not decision variables, but with the idea to automatize this codes they are defined as decision variables and then a constraint is added to set its values to a unique values

Doing this is a little automatice the creation of the constraints and open the posibility to transform the observed variables into a decision variables and connect other models/tanks/process to this observed variables with a less modification in the codes

In [ ]:
### get list of observed variables
list_observed_variables = config_allvariables[config_allvariables['clasification'] == 'O']['feature_name'].tolist()

### get table with observed variables and its values
config_initvalues_observed_variables = config_initvalues[config_initvalues['feature_name'].isin(list_observed_variables)]
config_initvalues_observed_variables = config_initvalues_observed_variables.reset_index().drop(columns = 'index') # reset index to count by index
config_initvalues_observed_variables

In [ ]:
for index_var in range(len(config_initvalues_observed_variables)):
    
    # get config values
    config_names_decision_var = config_initvalues_observed_variables.loc[index_var, 'feature_name']
    print('init values observed variables: ', config_names_decision_var)

    # add constraint observed variables fixed
    gppd.add_constrs(model_opt,
                     decision_var[config_names_decision_var],  # decision var
                     gp.GRB.EQUAL,
                     config_initvalues_observed_variables[config_initvalues_observed_variables['feature_name'] == config_names_decision_var]['init_values'].values[0],  # value observed variable
                     name = f'set value of observed variable: {config_names_decision_var}')

### 8. Load a Machine Learning Model as constraints that represent the relations in the process

In [ ]:
for index_modelml in range(len(config_modelsml)):

    ############ get the name (ID) of ml models ############
    config_names_modelsml = config_modelsml.loc[index_modelml, 'name_process_model']
    print('\nname model - model id: ', config_names_modelsml)


    ############ load artifact model ############
    model = models_ml[config_names_modelsml]
    type_model = type(model)

    
    ############ load parameters to build the machine learning models constraints ############
    # Get the list of features (with the SAME name used in the training of optimizer). and the SAME ORDER
    config_mapprocess_mlmodels_to_instance = config_mapprocess_mlmodels[config_mapprocess_mlmodels['name_process_model'] == config_names_modelsml]
    config_mapprocess_mlmodels_to_instance_features = config_mapprocess_mlmodels_to_instance[config_mapprocess_mlmodels_to_instance['clasificacion'] != 'T']
    list_features_instance = config_mapprocess_mlmodels_to_instance_features['feature_name'].tolist()
    
    # Get list of target
    config_mapprocess_mlmodels_to_instance_target = config_mapprocess_mlmodels_to_instance[config_mapprocess_mlmodels_to_instance['clasificacion'] == 'T']
    list_target_instance = config_mapprocess_mlmodels_to_instance_target['feature_name'].tolist()
    
    # Construct a list of targets of decision var acording the list of the target. # all the models predict only one target
    config_names_target_decision_var = list_target_instance[0]
    target_decision_var = decision_var[config_names_target_decision_var] 


    ############ build machine learning models constraints ############
    # identifity if the model loaded is a custom/rules or machine learning model
    if type(model) == pd.DataFrame:
        print('constraint custom model')
    
        # rename to remember the model is a dataframe with factors in custom model
        dataframe_model = model.copy()
        
        # for each feature in the model, get the decision var and multiply with its factor and build the left side constraint
        left_side_constraint = 0
        for index_feature_model in range(len(dataframe_model)):
        
            # get name of decision var
            config_names_decision_var = dataframe_model.loc[index_feature_model, 'feature_name']
            print('decision var: ', config_names_decision_var)
            
            # get factor of the decision var
            factor_decision_var = dataframe_model.loc[index_feature_model, 'factor_model']
            
            # multiply decision var with the factor and save in the left side constraint
            left_side_constraint += decision_var[config_names_decision_var] * factor_decision_var
    
    
        # define function as constraint
        gppd.add_constrs(model_opt, 
                         left_side_constraint,
                         gp.GRB.EQUAL, 
                         target_decision_var, # decision var target
                         name = f'function as constraint output predict {list_target_instance[0] }'
                        )
    
    else:
        print('constraint machine learning model')
    
        # Construct a list features of decision var acording the order in the list. 
        list_features_decision_var_instance = []
        for config_names_decision_var in list_features_instance:
            #print('feature - decision var append: ', config_names_decision_var)
            list_features_decision_var_instance.append(decision_var[config_names_decision_var])
        
        # Generate instance of machine learning model
        instance = pd.DataFrame(list_features_decision_var_instance).T
        instance.columns = list_features_instance
        
        # # Add machine learning model as constraint
        add_predictor_constr(gp_model = model_opt, 
                             predictor = model, 
                             input_vars = instance, # instance pandas gurobi
                             output_vars = target_decision_var, # decision var target
                             name = f'model_predict {list_target_instance[0]}'
                            )

### 9. Define objective optimization
Objetive that no generate infeasibility

In [ ]:
# optization
model_opt.setObjective(decision_var['Y1'].sum() + decision_var['Y2'].sum() + decision_var['Y3'].sum(),
                       gp.GRB.MAXIMIZE)

## ----> return the mdoel_opt that has defined for this instance can get the solution <----

In [ ]:
model_opt

### 10. Optimize and get optimal values

In [ ]:
# solve
model_opt.optimize()

In [ ]:
#### know the status of the model - 2 a optimal solution was founded
# docu: https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes
model_opt.Status

In [ ]:
# get optimal values and save in a dataframe
######## create a dataframe with set as index
solution = pd.DataFrame(index = index_set_time)

######################## save optimal values - features of models (only the features) ########################
solution["var_X1"] = decision_var['X1'].gppd.X
solution["var_O1"] = decision_var['O1'].gppd.X
solution["var_O2"] = decision_var['O2'].gppd.X
solution["var_O3"] = decision_var['O3'].gppd.X
solution["var_Y1"] = decision_var['Y1'].gppd.X
solution["var_O4"] = decision_var['O4'].gppd.X
solution["var_Z1"] = decision_var['Z1'].gppd.X
solution["var_X2"] = decision_var['X2'].gppd.X
solution["var_O5"] = decision_var['O5'].gppd.X
solution["var_O6"] = decision_var['O6'].gppd.X
solution["var_Y2"] = decision_var['Y2'].gppd.X
solution["var_Y3"] = decision_var['Y3'].gppd.X
solution["var_X3"] = decision_var['X3'].gppd.X
solution["var_O7"] = decision_var['O7'].gppd.X
solution["var_TL1"] = decision_var['TL1'].gppd.X
solution["var_TL2"] = decision_var['TL2'].gppd.X
solution["var_TL3"] = decision_var['TL3'].gppd.X

######################## # get value objetive function ########################
opt_objetive_function = model_opt.ObjVal

In [ ]:
# show value objetive function
opt_objetive_function

In [ ]:
# show value decision variables
solution.T